링크입니다.  
작업형 1-1 : https://www.kaggle.com/code/agileteam/3rd-type1-1-3-1-1  
작업형 1-2 : https://www.kaggle.com/code/agileteam/3rd-type1-2-3-1-2  
작업형 1-3 : https://www.kaggle.com/code/agileteam/3rd-type1-3-3-1-3  
작업형 2 : https://www.kaggle.com/code/agileteam/3rd-type2-3-2-baseline

In [58]:
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score, classification_report
from lightgbm import LGBMClassifier

### 작업형 1

In [9]:
# 2022년 데이터 중 2022년 중앙값보다 큰 값의 데이터 수

df1 = pd.read_csv("./data/csv/t1-data2.csv")

df1 = df1.T

df1.columns = df1.iloc[0,:]

df1.drop("year", axis=0, inplace=True)

df1.reset_index(drop=True, inplace=True)

med_2022 = df1["2022년"].median()

over_median_2022 = df1[df1["2022년"] > med_2022]

print("2022년 데이터 중 2022 중앙값보다 큰 값의 데이터 수 =", over_median_2022.shape[0])

2022년 데이터 중 2022 중앙값보다 큰 값의 데이터 수 = 50


In [15]:
# 결측치 데이터(행)을 제거하고, 앞에서부터 60% 데이터만 활용해, 'f1' 컬럼 3사분위 값을 구하시오
# 60%가 소수점일 경우 절사(예: 36.6 일때 36으로 계산)

df1 = pd.read_csv("./data/csv/t1-data1.csv")

df1.dropna(axis=0, how="any", inplace=True)

percent60 = math.floor(df1.shape[0] * 0.6)

# print(percent60)

df1_60 = df1.iloc[:percent60]

print("f1 컬럼의 3사분위 값", df1_60["f1"].quantile(0.75))

f1 컬럼의 3사분위 값 77.25


In [21]:
# 결측치가 제일 큰 값의 컬럼명을 구하시오

df1 = pd.read_csv("./data/csv/t1-data1.csv")

df1_null = df1.isnull().sum()

max_null = df1_null.max()

print("결측치가 제일 큰 값의 컬럼명 :",df1_null[df1_null==max_null].index[0])

결측치가 제일 큰 값의 컬럼명 : f1


### 작업형 2

여행 보험 패키지 상품을 구매할 확률 값을 구하시오  
- 예측할 값(y): TravelInsurance (여행보험 패지지를 구매 했는지 여부 0:구매안함, 1:구매)
- 평가: roc-auc 평가지표
- data: t2-1-train.csv, t2-1-test.csv

In [88]:
rdst = 22

trip_tr = pd.read_csv("./data/csv/t2-1-train.csv")
trip_tt = pd.read_csv("./data/csv/t2-1-test.csv")

# trip_tr.info()

len_tr = trip_tr.shape[0]

trip_tr_ob = trip_tr.select_dtypes("object")
trip_tt_ob = trip_tt.select_dtypes("object")


## 두 데이터를 합친 후 onehotencoding

trip_conc = pd.concat([trip_tr.drop("TravelInsurance", axis=1), trip_tt], axis=0)

trip_conc.drop("id", axis=1, inplace=True)

trip_onehot = pd.get_dummies(trip_conc, columns=["Employment Type"], dtype=int)

trip_onehot.replace({"Yes":1, "No":0}, inplace=True)


## 두 데이터를 다시 분리

train_X = trip_onehot.iloc[:len_tr]

test_X = trip_onehot.iloc[len_tr:]

train_y = trip_tr.iloc[:,-1]

# print(train_X.shape, test_X.shape, train_y.shape)

## 학습용 데이터와 검증용 데이터로 분리

# print(train_y.value_counts())    # 불균형 데이터

X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.2, stratify=train_y, random_state=rdst)

# print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

## scailing

mmsc = MinMaxScaler()

mmsc.fit(X_train)

X_train_sc = mmsc.transform(X_train)

X_val_sc = mmsc.transform(X_val)

test_X_sc = mmsc.transform(test_X)

# print(X_train_sc)

## 모델 학습

lgb = LGBMClassifier(n_estimators=50, max_depth=3)

lgb.fit(X_train_sc, y_train)

y_pred = lgb.predict(X_val_sc)

print(classification_report(y_val, y_pred))
print("검증 데이터를 통한 모델의 roc auc score =",round(roc_auc_score(y_val, y_pred), 4))


## test data로 보험을 들 확률 구하기

predict_prob = lgb.predict_proba(test_X_sc)

print(predict_prob[:,1])

              precision    recall  f1-score   support

           0       0.81      0.96      0.88       193
           1       0.90      0.57      0.70       105

    accuracy                           0.83       298
   macro avg       0.85      0.77      0.79       298
weighted avg       0.84      0.83      0.81       298

검증 데이터를 통한 모델의 roc auc score = 0.7676
[0.23192064 0.21934955 0.0566611  0.94695726 0.05083206 0.95836519
 0.22453826 0.19994051 0.82363617 0.19906553 0.19549988 0.07013622
 0.1752611  0.18373324 0.97491363 0.1949153  0.18008366 0.1726939
 0.22221805 0.22957023 0.09452332 0.17319926 0.9583993  0.93166024
 0.07013622 0.9581889  0.96073811 0.06410157 0.96161818 0.9581889
 0.20880259 0.20196929 0.21573446 0.22050872 0.14545479 0.81647332
 0.1767303  0.17724494 0.94695726 0.20196929 0.1726939  0.16718117
 0.19549988 0.07013622 0.95936426 0.10062919 0.95836519 0.19994051
 0.22221805 0.16718117 0.1726939  0.21854616 0.31509181 0.21545383
 0.07863404 0.06410157 0.1539033  